In [199]:
import pandas as pd
import re

In [200]:


import numpy as np
from collections import Counter
def post(x,topk,frc):
    #格式处理：c
#     x = np.array(x).flatten().tolist()
#     x=' '.join(x)
    x = x.split() 
    #过滤
#     r = []
#     for i in Counter(x).most_common(10000):
#         if i[1]>1:
#         #print(i)
#         r.append(i)
#     return r
    return[i[0] for i in  Counter(x).most_common(topk) if i[1]>frc and len(i[0])>1]


def make_df_entity_1(path,flag='train'):
    out_path = path+'.df_entity_1.csv'
    df1 = pd.read_csv(path,sep='\t')
    df1['top_esm_entity']=df1.esm_content.map(lambda x:post(x,topk=30,frc=2))
    df1['top_esm_title']=df1.esm_title.map(lambda x:post(x,topk=5,frc=0))



    newlines=[]
    for i,line in df1.iterrows():


        texta =line['content']
        
        if flag=='test':
            entity_real=['null']
        else:
            entity_real = line['entity'].split()
        entity_recall = line['top_esm_title'] + line['top_esm_entity']
        for entity in entity_recall:
            textb = entity
            if entity in entity_real:
                label = 1
            else:
                label = 0
            newlines.append([line['newsId'],texta,textb,label])
    df_entity = pd.DataFrame(newlines,columns=['newsId','texta','textb','label'])
    df_entity.to_csv(out_path,index=False,sep='\t')




In [276]:
def make_df_entity_2(path,flag='train'):
    agg = True
    if agg:
        out_path = path+'.agg.df_entity_2.csv'
    else:
        out_path = path+'.df_entity_2.csv'
    print(out_path)
    df1 = pd.read_csv(path,sep='\t')
    df1['top_esm_entity']=df1.esm_content.map(lambda x:post(x,topk=30,frc=2))
    df1['top_esm_title']=df1.esm_title.map(lambda x:post(x,topk=5,frc=0))

    newlines=[]
    for i,line in df1.iterrows():

        entity_recall = line['top_esm_title'] + line['top_esm_entity']
        sentences = re.split(r'。|？|！', line['content'])
        if flag =='test':
            entity_real=['null']
        else:
            entity_real = line['entity'].split()
        
        entity_real = [clean_texts(x) for x in entity_real]
        for sentence in sentences:
            sentence = clean_texts(sentence)
            for j ,entity in enumerate(entity_recall):

                if entity[0] in ['*','+']:
                    entity='\\'+entity
                if  entity=='c++':
                    entity='c\++'

                textb = entity
                if entity in entity_real:
                    label = 1
                else:
                    label = 0
                try:
                    if re.search(entity,sentence):
                        newlines.append([line['newsId'],sentence,entity,label])
                except:
                    pass
                    #print(entity,sentence)


    df_entity = pd.DataFrame(newlines,columns=['newsId','texta','textb','label'])
    #df_entity.dropna(inplace=True)
    
    if agg:
        gg = df_entity.groupby(['newsId','textb'])
        df_agg2 = gg['texta'].apply(lambda x:','.join(x)).reset_index()
        df_agg2['label'] = gg['label'].apply(list).reset_index()['label']
        df_agg2['label'] = df_agg2['label'].map(lambda x:x[0])

        df_entity=df_agg2



    df_entity.dropna(inplace=True)

    print(df_entity.head())
    df_entity.to_csv(out_path,index=False,sep='\t')




In [277]:
import re
def clean_texts(x):
        
        #return  re.sub("[+——()?（）\\n \\t ]+", "",x)
        return re.sub("[\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）]+", "",x)


In [278]:
path = '/sdb1/zhangle/2019/zhangle11/sohu-2019/data/coreEntityEmotion_train.txt.esm_entity.csv'
#make_df_entity_1(path)
make_df_entity_2(path)

path2 = '/sdb1/zhangle/2019/zhangle11/sohu-2019/data/coreEntityEmotion_test_stage1.txt.esm_entity.csv'
#make_df_entity_1(path2,'test')
make_df_entity_2(path2,'test')


/sdb1/zhangle/2019/zhangle11/sohu-2019/data/coreEntityEmotion_train.txt.esm_entity.csv.agg.df_entity_2.csv
     newsId textb                                              texta  label
0  0000214b    12  朋友圈大神解读如下现在假设你的采购成本是100元成交价是120元,如果按17增值税率你的进项...      0
1  0000214b    16  蓝字文章来源：服装邦3月15日两会答记者问总理透露今年下决心要进行更大规模的减税降费4月1日...      0
2  0000214b    价格  我们抛开企业竞争力量之说仅就降税了对企业的价格是怎么影响的来给提出降价要求的企业和应对降价要...      0
3  0000214b    企业  这本是对企业的一项重大利好,这个客户说降一个点还可以接受现在是降三个点客户是上市公司强势去年...      0
4  0000214b    减税  蓝字文章来源：服装邦3月15日两会答记者问总理透露今年下决心要进行更大规模的减税降费4月1日...      0
/sdb1/zhangle/2019/zhangle11/sohu-2019/data/coreEntityEmotion_test_stage1.txt.esm_entity.csv.agg.df_entity_2.csv
     newsId textb                                              texta  label
0  0000327e    价格  一国内焦亚硫酸钠价格走势图据生意社监测本周国内焦亚硫酸钠稳定前行工业级焦亚硫酸钠周初均价19...      0
1  0000327e    企业  焦亚硫酸钠行业年后生产较为稳定企业整体库存尚可多以完成老客户订单为主新单增加有限交投稍显清淡...      0
2  0000327e    市场  二行情分析产品：本周焦亚硫酸钠市场整体行情相对稳定前行工业级焦亚硫酸钠市场价格区间1850-...      0
3  0000327e    整体  二

In [286]:
p = '/sdb1/zhangle/2019/zhangle11/sohu-2019/data/coreEntityEmotion_train.txt.esm_entity.csv'
df=pd.read_csv(p,sep='\t')

In [288]:
df.content.map(lambda x:len(x)).mean()

1293.505175

In [283]:
df.shape

(686369, 4)

In [284]:
df.head()

,newsId,textb,texta,label
0,0000214b,12,"朋友圈大神解读如下现在假设你的采购成本是100元成交价是120元,如果按17增值税率你的进项...",0
1,0000214b,16,蓝字文章来源：服装邦3月15日两会答记者问总理透露今年下决心要进行更大规模的减税降费4月1日...,0
2,0000214b,价格,我们抛开企业竞争力量之说仅就降税了对企业的价格是怎么影响的来给提出降价要求的企业和应对降价要...,0
3,0000214b,企业,"这本是对企业的一项重大利好,这个客户说降一个点还可以接受现在是降三个点客户是上市公司强势去年...",0
4,0000214b,减税,蓝字文章来源：服装邦3月15日两会答记者问总理透露今年下决心要进行更大规模的减税降费4月1日...,0


In [194]:
p = '/sdb1/zhangle/2019/zhangle11/sohu-2019/data/coreEntityEmotion_train.txt.esm_entity.csv.df_entity_1.csv'
df=pd.read_csv(p,sep='\t',nrows=100)

In [195]:
df.head()

,newsId,texta,textb,label
0,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,sia,0
1,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,智慧,0
2,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,智慧工厂,0
3,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,机器,0
4,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,智能,0


In [37]:
path = '/sdb1/zhangle/2019/zhangle11/sohu-2019/data/coreEntityEmotion_train.txt.esm_entity.csv'

In [38]:
df1 = pd.read_csv(path,sep='\t')

In [39]:
df1.head(100)

,newsId,content,title,entity,emotion,esm_title,esm_content
0,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,sia智慧工厂展，誉洋以“智”取胜,3d 工业 机器视觉,POS POS POS,sia 智慧 智慧工厂,上海 上海国 国际 工业 工业自动化 机器 机器人 ...
1,cb8e8b79,大数据之热度，已无需多言。业内众多关于大数据可视化应用领域的声音与讨论，大多集中在行业领域和...,大数据可视化应用领域探析,数据 卫星 可视化,POS POS POS,大数 大数据 应用 领域,大数 大数据 热度 业内 内众 ...
2,f85c18e2,过年期间和一些搞运输的老朋友聊天，大家始终绕不开的一个话题：“如今物流行业越来越难干了！”想...,大车发大财的时代一去不返？货运行业还有春天吗？,物流 货运 信息,POS NEG POS,大车 时代 货运 行业 还 春天...,年期 期间 老朋友 聊天 大家 ...
3,889199b4,水泥路面起砂如何处理？修补后材料会发生脱落吗？水泥路面起砂病害的发生可能是材料老化的现象，也...,水泥路面起砂如何处理？修补后材料会发生脱落吗？,路面 水泥 病害,NORM NORM NORM,水泥 路面 材料,水泥 路面 材料 ...
4,a811d20e,我们今天再开始复盘之前，先简单说下行情，昨天大幅度波动90点的时候，很多人看空，但是却没有勇...,股市高位还将继续涨， 这几类股票有望成为新热点,阳线 股票 涨势,POS POS POS,股 股市 高 高位 还 继 类股 股票 热点,复盘 下行 行情 ...
5,4e798999,对于大部分入门玩家来说，在电源的选择上往往都比较随意。他们在意的方面一般只有功率的大小，而对...,装机容易忽视的组件，主机电源不能随便选,evga 电源 组件,NORM NORM NORM,装机 组件 主机 机电 电源,大部 大部分 入门 玩 玩家 电源 ...
6,1f5679fb,新打的水泥地面受冻掉皮怎么修复？水泥路面、地面掉皮病害发生后，有一些水泥路面、地面发生病害后...,新打的水泥地面受冻掉皮怎么修复？修补质量如何保证？,水泥 病害 路面,NORM NORM NORM,水泥 泥地 地面 质量 ...,水泥 泥地 地面 水泥 路面 ...
7,1a8339ea,一、信托融资概念及特点\n信托是以资产为核心，信任为基础，委托为方式的财产管理制度。信托融资...,信托融资模式下房地产信托融资及发展趋势解析,信托 房地产 股权,NORM NORM NORM,信托 融资 模式 房地 房地产 信托 融资 发展...,信托 融资 概念 特点 信托 资产 核...
8,51203867,当“租购并举”成为国策，住房租赁的庞大市场需求和政策红利的持续加持，让越来越多人发现并逐步重...,地产基金投资公寓资产，抓好这两点是关键,租购并举 公寓资产 租客,POS NORM NORM,地产 地产基金 金投 基金投资 公寓 资产 关键,当 国策 住房 房租 大市 大市...
9,5bb9047a,随着生活压力和工作压力都不断增加的情况下，有心理问题的人也越来越多，很多人都开始求助心理咨询...,心理咨询小程序开发案例分析,心理 心理咨询师 小程序开发,NORM NORM POS,心理 心理咨询 小程 小程序 小程序开发 发案 案例,生活 压力 工作 工作压力 加 情况 ...


In [143]:
df1['top_esm_entity']=df1.esm_content.map(lambda x:post(x,topk=30,frc=2))

In [144]:
df1['top_esm_title']=df1.esm_title.map(lambda x:post(x,topk=5,frc=0))

In [145]:
df1.head(30)

,newsId,content,title,entity,emotion,esm_title,esm_content,top_esm_entity,top_in_cnt,entity_cnt,top_esm_title
0,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,sia智慧工厂展，誉洋以“智”取胜,3d 工业 机器视觉,POS POS POS,sia 智慧 智慧工厂,上海 上海国 国际 工业 工业自动化 机器 机器人 ...,"[机器, 智能, 工业, 企业, 机器视觉, 3d, 机器人, 展会, 工业自动化, 技术,...",3,3,"[sia, 智慧, 智慧工厂]"
1,cb8e8b79,大数据之热度，已无需多言。业内众多关于大数据可视化应用领域的声音与讨论，大多集中在行业领域和...,大数据可视化应用领域探析,数据 卫星 可视化,POS POS POS,大数 大数据 应用 领域,大数 大数据 热度 业内 内众 ...,"[智能, 数据, 领域, 应用, 大数, 大数据, 决策, 卫星, 态势, 商业, 设备, ...",2,3,"[大数, 大数据, 应用, 领域]"
2,f85c18e2,过年期间和一些搞运输的老朋友聊天，大家始终绕不开的一个话题：“如今物流行业越来越难干了！”想...,大车发大财的时代一去不返？货运行业还有春天吗？,物流 货运 信息,POS NEG POS,大车 时代 货运 行业 还 春天...,年期 期间 老朋友 聊天 大家 ...,"[物流, 货运, 信息, 行业, 服务, 顺风, 货源, 货主, 同城, 大车, 道路, 资...",3,3,"[大车, 时代, 货运, 行业]"
3,889199b4,水泥路面起砂如何处理？修补后材料会发生脱落吗？水泥路面起砂病害的发生可能是材料老化的现象，也...,水泥路面起砂如何处理？修补后材料会发生脱落吗？,路面 水泥 病害,NORM NORM NORM,水泥 路面 材料,水泥 路面 材料 ...,"[路面, 水泥, 水平, 材料, 病害, 原因, 可能, 现象, 问题, 表层, 粘结力]",3,3,"[水泥, 路面, 材料]"
4,a811d20e,我们今天再开始复盘之前，先简单说下行情，昨天大幅度波动90点的时候，很多人看空，但是却没有勇...,股市高位还将继续涨， 这几类股票有望成为新热点,阳线 股票 涨势,POS POS POS,股 股市 高 高位 还 继 类股 股票 热点,复盘 下行 行情 ...,"[上涨, 就是, 市场, 趋势, 行业, 股票, 阳线, 前期, 个股, 消费, 涨势]",1,3,"[股市, 高位]"
5,4e798999,对于大部分入门玩家来说，在电源的选择上往往都比较随意。他们在意的方面一般只有功率的大小，而对...,装机容易忽视的组件，主机电源不能随便选,evga 电源 组件,NORM NORM NORM,装机 组件 主机 机电 电源,大部 大部分 入门 玩 玩家 电源 ...,"[电源, ev, ga, 功率, g3, 硬件, 电脑, 大功, 大功率, 金牌, 转化率,...",2,3,"[装机, 组件, 主机, 机电, 电源]"
6,1f5679fb,新打的水泥地面受冻掉皮怎么修复？水泥路面、地面掉皮病害发生后，有一些水泥路面、地面发生病害后...,新打的水泥地面受冻掉皮怎么修复？修补质量如何保证？,水泥 病害 路面,NORM NORM NORM,水泥 泥地 地面 质量 ...,水泥 泥地 地面 水泥 路面 ...,"[水泥, 路面, 病害, 地面, 泥地, 冻害, 问题, 可能, 冬季, 一次性, 质量, ...",3,3,"[水泥, 泥地, 地面, 质量, 保证]"
7,1a8339ea,一、信托融资概念及特点\n信托是以资产为核心，信任为基础，委托为方式的财产管理制度。信托融资...,信托融资模式下房地产信托融资及发展趋势解析,信托 房地产 股权,NORM NORM NORM,信托 融资 模式 房地 房地产 信托 融资 发展...,信托 融资 概念 特点 信托 资产 核...,"[信托, 房地, 房地产, 融资, 模式, 资金, 信托公司, 企业, 项目, 股权, 投资...",2,3,"[信托, 融资, 模式, 房地, 房地产]"
8,51203867,当“租购并举”成为国策，住房租赁的庞大市场需求和政策红利的持续加持，让越来越多人发现并逐步重...,地产基金投资公寓资产，抓好这两点是关键,租购并举 公寓资产 租客,POS NORM NORM,地产 地产基金 金投 基金投资 公寓 资产 关键,当 国策 住房 房租 大市 大市...,"[数据, 公寓, 投资, 资金, 智能, 租客, 问题, 管理, 水滴, 运营, 行业, 金...",1,3,"[地产, 地产基金, 金投, 基金投资, 公寓]"
9,5bb9047a,随着生活压力和工作压力都不断增加的情况下，有心理问题的人也越来越多，很多人都开始求助心理咨询...,心理咨询小程序开发案例分析,心理 心理咨询师 小程序开发,NORM NORM POS,心理 心理咨询 小程 小程序 小程序开发 发案 案例,生活 压力 工作 工作压力 加 情况 ...,"[心理, 心理咨询, 用户, 问题, 小程, 小程序, 平台, 疾病, 服务, 心理医生, ...",2,3,"[心理, 心理咨询, 小程, 小程序, 小程序开发]"


In [146]:
df1.apply(lambda x:len(set(x['top_esm_entity']+x['top_esm_title'])),axis=1).sum()

718038

In [147]:
df1['top_in_cnt'] = df1.apply(lambda x:len(set(x['top_esm_entity']+x['esm_title'].split()) & set(x['entity'].split())),axis=1)

In [148]:
df1['entity_cnt'].sum()

87337

In [149]:
df1['entity_cnt'] = df1['entity'].apply(lambda x:len(x.split()))

In [150]:
float(df1.top_in_cnt.sum())/float(df1.entity_cnt.sum())

0.7580177931460893

#

词频>0:0.86    
词频>1:0.8022945601520547    
    0.76   1277422
    
top 30:    0.75   922546


content 
top =30 frc=1:    0.782   993440    
top =10 frc=1:    0.729   487022    

top =10 frc=1 top =5 frc=0:    0.729   423132      
top =10 frc=2 top =5 frc=0:    0.724   400176      
 top =30 frc=2 top =5 frc=0:    0.758                   718038                

In [151]:
df1.head()

,newsId,content,title,entity,emotion,esm_title,esm_content,top_esm_entity,top_in_cnt,entity_cnt,top_esm_title
0,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,sia智慧工厂展，誉洋以“智”取胜,3d 工业 机器视觉,POS POS POS,sia 智慧 智慧工厂,上海 上海国 国际 工业 工业自动化 机器 机器人 ...,"[机器, 智能, 工业, 企业, 机器视觉, 3d, 机器人, 展会, 工业自动化, 技术,...",3,3,"[sia, 智慧, 智慧工厂]"
1,cb8e8b79,大数据之热度，已无需多言。业内众多关于大数据可视化应用领域的声音与讨论，大多集中在行业领域和...,大数据可视化应用领域探析,数据 卫星 可视化,POS POS POS,大数 大数据 应用 领域,大数 大数据 热度 业内 内众 ...,"[智能, 数据, 领域, 应用, 大数, 大数据, 决策, 卫星, 态势, 商业, 设备, ...",2,3,"[大数, 大数据, 应用, 领域]"
2,f85c18e2,过年期间和一些搞运输的老朋友聊天，大家始终绕不开的一个话题：“如今物流行业越来越难干了！”想...,大车发大财的时代一去不返？货运行业还有春天吗？,物流 货运 信息,POS NEG POS,大车 时代 货运 行业 还 春天...,年期 期间 老朋友 聊天 大家 ...,"[物流, 货运, 信息, 行业, 服务, 顺风, 货源, 货主, 同城, 大车, 道路, 资...",3,3,"[大车, 时代, 货运, 行业]"
3,889199b4,水泥路面起砂如何处理？修补后材料会发生脱落吗？水泥路面起砂病害的发生可能是材料老化的现象，也...,水泥路面起砂如何处理？修补后材料会发生脱落吗？,路面 水泥 病害,NORM NORM NORM,水泥 路面 材料,水泥 路面 材料 ...,"[路面, 水泥, 水平, 材料, 病害, 原因, 可能, 现象, 问题, 表层, 粘结力]",3,3,"[水泥, 路面, 材料]"
4,a811d20e,我们今天再开始复盘之前，先简单说下行情，昨天大幅度波动90点的时候，很多人看空，但是却没有勇...,股市高位还将继续涨， 这几类股票有望成为新热点,阳线 股票 涨势,POS POS POS,股 股市 高 高位 还 继 类股 股票 热点,复盘 下行 行情 ...,"[上涨, 就是, 市场, 趋势, 行业, 股票, 阳线, 前期, 个股, 消费, 涨势]",3,3,"[股市, 高位]"


In [156]:


newlines=[]
for i,line in df1.head().iterrows():


    texta =line['content']
    entity_real = line['entity'].split()
    entity_recall = line['top_esm_title'] + line['top_esm_entity']
    for entity in entity_recall:
        textb = entity
        if entity in entity_real:
            label = 1
        else:
            label = 0
        newlines.append([line['newsId'],texta,textb,label])
df_pair = pd.DataFrame(newlines,columns=['newsId','texta','textb','label'])

In [181]:
gg = df_pair[df_pair.label==1][['newsId','textb']].groupby(['newsId'])
df_agg2 = gg['textb'].apply(lambda x:','.join(x)).reset_index()
df_agg2['textb']=df_agg2['textb'].map(lambda x:[i[0] for i in Counter(x.split()).most_common(3)])
df_agg2['textb']=df_agg2['textb'].apply(lambda x:','.join(x))


In [183]:
df_pair.head()

,newsId,texta,textb,label
0,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,sia,0
1,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,智慧,0
2,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,智慧工厂,0
3,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,机器,0
4,4e36d02a,第十七届上海国际工业自动化及机器人展与上海智能工厂展览会于2019年3月1日圆满落下帷幕。展...,智能,0
